In [1]:
from surprise import Dataset
import numpy as np
import matplotlib.pyplot as plt
from surprise.model_selection import KFold
from surprise import accuracy
import pandas as pd
from surprise import Dataset, evaluate, Reader, accuracy, Trainset
from pathlib import Path

In [2]:
import os.path
from random import *

In [7]:
def inteNameBis(i):
    if i==0:
        return "1-9"
    else :
        return str(int(i*10))+"-"+str(int(i*10+9))

In [8]:
for fold in range(10):
    print("______________________________________________________________")
    print("fold : ",fold)
    print("load nb ratings")
    arDico=pd.read_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv').to_numpy()
    d={}
    for i in range(len(arDico)) :
        d[arDico[i,0]]=arDico[i,1]
    del arDico
    
    print("load testing dataset")
    arTest=pd.read_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip').to_numpy()
    size=len(arTest)
    dicoDf={}
    print("Repartition Start")
    for j in range(size): 
        #print(arTest[j,0],d[int(arTest[j,0])])
        nb=d[int(arTest[j,0])]//10
        if nb in dicoDf :
            dicoDf[nb].append([arTest[j]])
        else:
            dicoDf[nb]=list()
            dicoDf[nb].append([arTest[j]])
    del d
    
    print("Sauvegarde des intervalles")
    for j in dicoDf:
        a=np.concatenate(dicoDf[j],axis=0)
        dfTemp = pd.DataFrame({'userId':a[:,0],'movieId':a[:,1],'rating':a[:,2]})
        dfTemp.to_parquet('dataCV3/df'+inteNameBis(j)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
        del dfTemp,a
    
    print("Creation of the large intervalles")
    for i in range(10,50,5):
        listDF=list()
        for j in range(5):
            listDF.append(pd.read_parquet('dataCV3/df'+inteNameBis(i+j)+'_fold_'+str(fold)+'.gzip'))
        dfTemp=pd.concat(listDF)
        dfTemp.to_parquet('dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    
    print("Creation of the intervalle sup500")
    listDF=list()
    for i in range(50,800):
        path='dataCV3/df'+inteNameBis(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            listDF.append(pd.read_parquet(path))
    dfTemp=pd.concat(listDF)
    dfTemp.to_parquet('dataCV3/df'+inteName(50)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del dfTemp,listDF
    
    print("Creation of the intervalle inf20")
    df1=pd.read_parquet('dataCV3/df1-9'+'_fold_'+str(fold)+'.gzip')
    df2=pd.read_parquet('dataCV3/df10-19'+'_fold_'+str(fold)+'.gzip')
    dfInf20=pd.concat([df1,df2])
    dfInf20.to_parquet('dataCV3/dfInf20'+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del df1,df2,dfInf20
    
    print("Creation of the intervalle sup20")
    listDF=list()
    for i in range(2,10):
        listDF.append(pd.read_parquet('dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'))
    for i in range(10,51,5):
        listDF.append(pd.read_parquet('dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'))
    dfSup20=pd.concat(listDF)
    dfSup20.to_parquet('dataCV3/dfSup20_fold_'+str(fold)+'.gzip', compression='gzip',index=False)    
    del dfSup20,listDF,dicoDf

______________________________________________________________
fold :  0
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  1
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  2
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  3
load nb ratings
load testing dataset
Repartition Start
Sauve

In [2]:
from surprise import SVD
from surprise import SVDpp
from surprise import BaselineOnly
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore

In [3]:
dicoDfTest={}
for fold in range(10):
    dicoDfTest[fold]=pd.read_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip')

In [4]:
def listTestCV(fold) :    
    testInte=list()
    reader = Reader()
    dfTest=pd.read_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip')
    arTest=dfTest.to_numpy()
    data=Dataset.load_from_df(dfTest,reader) 
    testInte.append(data.build_full_trainset().build_testset())

    df=pd.read_parquet('dataCV3/dfInf20_fold_'+str(fold)+'.gzip')
    data=Dataset.load_from_df(df,reader) 
    testInte.append(data.build_full_trainset().build_testset())

    df=pd.read_parquet('dataCV3/dfSup20_fold_'+str(fold)+'.gzip')
    data=Dataset.load_from_df(df,reader) 
    testInte.append(data.build_full_trainset().build_testset())

    for j in range(10):
        my_file = Path("dataCV3/df"+inteName(j)+'_fold_'+str(fold)+'.gzip')
        df=pd.read_parquet(my_file)
        data=Dataset.load_from_df(df,reader) 
        testInte.append(data.build_full_trainset().build_testset())
    for j in range(10,51,5):
        my_file = Path("dataCV3/df"+inteName(j)+'_fold_'+str(fold)+'.gzip')
        df=pd.read_parquet(my_file)
        data=Dataset.load_from_df(df,reader) 
        testInte.append(data.build_full_trainset().build_testset()) 
    return testInte

In [5]:
def inteName(i):
    if i==0:
        return "1-9"
    elif i<10:
        return str(int(i*10))+"-"+str(int(i*10+9))
    elif i>=50:
        return 'sup500'
    else :
        return str(int(i*10))+"-"+str(int(i*10+49))

In [6]:
def trainsetCV(fold):
    dfTrain=pd.read_parquet('dataCV3/dfTrain_fold_'+str(fold)+'.gzip')
    reader = Reader()
    data=Dataset.load_from_df(dfTrain,reader)
    train=data.build_full_trainset()
    return train

In [7]:
def algoSVD() :
    algo=SVD()
    return algo
    
def algoSVDpp() :
    algo=SVDpp()
    return algo

def algoNMF() :
    algo=NMF()
    return algo

def algoBaselineOnly() :
    algo=BaselineOnly()
    return algo

def algoCoClustering() :
    algo=CoClustering()
    return algo

def algoNormalPredictor() :
    algo=NormalPredictor()
    return algo

def algoSlopeOne() :
    algo=SlopeOne()
    return algo

def algoKNNBasic() :
    algo=KNNBasic()
    return algo

def algoKNNWithMeans() :
    algo=KNNWithMeans()
    return algo

def algoKNNBaseline() :
    algo=KNNBaseline()
    return algo

def algoKNNWithZScore() :
    algo=KNNWithZScore()
    return algo

In [8]:
algoChoice = {"SVD" : algoSVD,
              "SVDpp" : algoSVDpp,
              "NMF"  : algoNMF,
              "BaselineOnly" : algoBaselineOnly,
              "CoClustering" : algoCoClustering,
              "NormalPredictor" : algoNormalPredictor,
              "SlopeOne" : algoSlopeOne,
              "KNNBasic" : algoKNNBasic,
              "KNNWithMeans" : algoKNNWithMeans,
              "KNNBaseline" : algoKNNBaseline,
              "KNNWithZScore" : algoKNNWithZScore}

In [9]:
algoListe=["BaselineOnly", "SVD", "NMF", "SlopeOne", 
           "CoClustering", "NormalPredictor", "SVDpp",
           "KNNBasic", "KNNWithMeans", "KNNBaseline", "KNNWithZScore"]

In [10]:
def ajouterResultat(algo,res,listTest,algoName):
    rmse=list()
    mae=list()
    for i in listTest:
        predictions = algo.test(i)
        rmse.append(accuracy.rmse(predictions, verbose=False))
        mae.append(accuracy.mae(predictions, verbose=False))
    res[algoName]=rmse+mae

In [11]:
def listPredictions(ar,algo):
    l=list()
    t=len(ar)
    for i in range(t):
        l.append(algo.predict(ar[i,0],ar[i,1],verbose=False)[3])
    return l

In [12]:
results={}

In [13]:
for name in algoListe[2:6] :
    print("--==  ",name,"  ==--")
    for fold in range(5): 
        print("tour ",fold," de cross val")
        algo=algoChoice[name]()
        print("-- Train --")
        train=trainsetCV(fold)
        algo.fit(train)
        print("-- Test --")
        ajouterResultat(algo,results,listTestCV(fold),name+str(fold))
        dicoDfTest[fold]["Predicted ratings "+name]=listPredictions(dicoDfTest[fold].to_numpy(),algo)

--==   NMF   ==--
tour  0  de cross val
-- Train --
-- Test --
tour  1  de cross val
-- Train --
-- Test --
tour  2  de cross val
-- Train --
-- Test --
tour  3  de cross val
-- Train --
-- Test --
tour  4  de cross val
-- Train --
-- Test --


MemoryError: 

In [14]:
lRMSEname=list()
lMAEname=list()
for i in range(10):
    lRMSEname.append("RMSE_"+inteName(i))
    lMAEname.append("MAE_"+inteName(i))
for i in range(10,51,5):
    lRMSEname.append("RMSE_"+inteName(i))
    lMAEname.append("MAE_"+inteName(i))

nameColumms=["algoName","RMSE_All","RMSE_<20","RMSE_20+"]+lRMSEname+["MAE_All","MAE_<20","MAE_20+"]+lMAEname

In [15]:
dfRes=pd.DataFrame(columns =nameColumms)
for key in results:
    dfRes.loc[len(dfRes)]=[key]+results[key]
dfRes.to_csv('data/resultRMSEbyInterval_1.csv',index=False)

In [16]:
for i in dicoDfTest :
    dicoDfTest[i].to_csv('data/prediction_2_fold_'+str(i)+'.csv',index=False)

MemoryError: 

In [24]:
df=pd.concat(dicoDfTest.values())
df.to_csv('data/prediction_full.csv',index=False)

C:\Users\vicbr\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
